In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb

from alpha_vantage.timeseries import TimeSeries
# from sqlalchemy import create_engine
ts = TimeSeries(key='<Put Your API Key Here>', output_format='pandas')

In [2]:
data, meta_data = ts.get_daily(symbol='ET', outputsize='full')
# 🔹 Rename columns for clarity
data.columns = ["Open", "High", "Low", "Close", "Volume"]

# 🔹 Convert index to datetime format
data.index = pd.to_datetime(data.index)

# 🔹 Convert index (date) into a regular column
data.reset_index(inplace=True)

# Calculate 100-day moving average for Close price
# data["100d_MA"] = data["Close"].rolling(window=100).mean()

# 🔹 Rename the date column
data.rename(columns={"date": "Date"}, inplace=True)
data['Date'].max()

Timestamp('2025-03-27 00:00:00')

In [7]:
data.head(5)

,Date,Open,High,Low,Close,Volume
0,2025-03-27,18.900,19.04,18.7500,18.88,9860399.0
1,2025-03-26,18.900,19.12,18.8550,18.93,10413196.0
2,2025-03-25,18.870,19.13,18.8100,18.88,8733757.0
3,2025-03-24,18.805,19.11,18.7729,18.88,12855173.0
4,2025-03-21,18.820,18.82,18.4500,18.60,15140374.0


In [13]:
con = duckdb.connect(database='./vangaurd.duckdb', read_only=False)

In [16]:
con.execute('DROP TABLE IF EXISTS stocks;')
con.register('temp_df', data)  # Register the DataFrame as a virtual table
con.execute("CREATE TABLE stocks AS SELECT * FROM temp_df")  # Create the table
con.unregister('temp_df') #unregister the temp table.

con.execute('SELECT max(Date) as last_date from stocks;').fetchone()


(datetime.datetime(2025, 3, 27, 0, 0),)

In [ ]:
con.close()